In [1]:
import xarray as xr
import pandas as pd
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import numpy as np
from glob import glob
from math import pi,exp

In [17]:
def coarsen(ci,weights):
    (times,ens,sn,we) = ci.shape
    sn_int = 57
    we_int = 41
    crs_atint = np.zeros((times,ens,sn_int,we_int))
    for t in range(times):
        for e in range(ens):
            frame = ci[t,e,:,:]
            dbuf = np.full((sn+6,we+6),np.nan)
            ones = np.full((sn+6,we+6),np.nan)
            crs = np.zeros_like(frame)
            for i in range(sn):
                dbuf[i+3][3:-3] = frame[i]
                ones[i+3][3:-3] = np.ones_like(frame[i])
            for i in range(0,sn,4):
                for j in range(0,we,4):
                    ci_sum = np.nansum(np.multiply(dbuf[i:i+7,j:j+7],weights))
                    weight_sum = np.nansum(np.multiply(ones[i:i+7,j:j+7],weights))
                    crs[i,j] = ci_sum/weight_sum
            for y in range(sn_int):
                for x in range(we_int):
                    crs_atint[t,e,y,x] = crs[4*y,4*x]
    return crs_atint

In [37]:
# create mask (Gaussian)
m = np.zeros((7,7))
var = 2
for i in range(7):
    for j in range(7):
        m[i,j] = exp(-((i-3)**2+(j-3)**2)/(2*var))
m = m/m[3,3]
print(m)

[[0.011109   0.03877421 0.082085   0.10539922 0.082085   0.03877421
  0.011109  ]
 [0.03877421 0.13533528 0.2865048  0.36787944 0.2865048  0.13533528
  0.03877421]
 [0.082085   0.2865048  0.60653066 0.77880078 0.60653066 0.2865048
  0.082085  ]
 [0.10539922 0.36787944 0.77880078 1.         0.77880078 0.36787944
  0.10539922]
 [0.082085   0.2865048  0.60653066 0.77880078 0.60653066 0.2865048
  0.082085  ]
 [0.03877421 0.13533528 0.2865048  0.36787944 0.2865048  0.13533528
  0.03877421]
 [0.011109   0.03877421 0.082085   0.10539922 0.082085   0.03877421
  0.011109  ]]


In [39]:
res_files = glob('2014*Z.nc')
we_int = np.arange(240.0,281.0,dtype=np.float32)
sn_int = np.arange(32.0,89.0,dtype=np.float32)
for res in res_files:
    print(res)
    res_ds = xr.open_dataset(res)
    res_ci = res_ds.data_vars['ci']
    res_ds.close()
    ci_crs = coarsen(res_ci.values,m)
    df_crs = xr.DataArray(ci_crs,
                    coords=[res_ci.time.values,
                            np.array(res_ci.ensemble_number.values,dtype=int),
                            sn_int,we_int],
                    dims=['time','ensemble_number','south_north',
                           'west_east'],
                    name='ci')
    name = res[:-3]+'_crs_gauss.nc'
    df_crs.to_netcdf(name)

20140402_1630Z.nc
20140402_1645Z.nc
20140402_1700Z.nc
20140402_1715Z.nc
20140402_1730Z.nc
20140402_1745Z.nc
20140402_1800Z.nc
20140402_1830Z.nc
20140402_1845Z.nc
20140402_1900Z.nc
20140402_1915Z.nc
20140402_1930Z.nc
20140402_1945Z.nc
20140402_2000Z.nc
20140402_2015Z.nc
20140402_2030Z.nc
20140402_2100Z.nc
20140402_2130Z.nc
20140402_2145Z.nc
20140402_2200Z.nc
20140402_2215Z.nc
20140402_2230Z.nc
20140405_1630Z.nc
20140405_1645Z.nc
20140405_1700Z.nc
20140405_1715Z.nc
20140405_1730Z.nc
20140405_1745Z.nc
20140405_1800Z.nc
20140405_1830Z.nc
20140405_1845Z.nc
20140405_1900Z.nc
20140405_1915Z.nc
20140405_1930Z.nc
20140405_1945Z.nc
20140405_2000Z.nc
20140405_2015Z.nc
20140405_2030Z.nc
20140405_2100Z.nc
20140405_2130Z.nc
20140405_2145Z.nc
20140405_2200Z.nc
20140405_2215Z.nc
20140405_2230Z.nc
20140409_1630Z.nc
20140409_1645Z.nc
20140409_1700Z.nc
20140409_1715Z.nc
20140409_1730Z.nc
20140409_1745Z.nc
20140409_1800Z.nc
20140409_1830Z.nc
20140409_1845Z.nc
20140409_1900Z.nc
20140409_1915Z.nc
20140409_1

20140521_1715Z.nc
20140521_1730Z.nc
20140521_1745Z.nc
20140521_1800Z.nc
20140521_1830Z.nc
20140521_1845Z.nc
20140521_1900Z.nc
20140521_1915Z.nc
20140521_1930Z.nc
20140521_1945Z.nc
20140521_2000Z.nc
20140521_2015Z.nc
20140521_2030Z.nc
20140521_2100Z.nc
20140521_2130Z.nc
20140521_2145Z.nc
20140521_2200Z.nc
20140521_2215Z.nc
20140521_2230Z.nc
20140522_1630Z.nc
20140522_1645Z.nc
20140522_1700Z.nc
20140522_1715Z.nc
20140522_1745Z.nc
20140522_1800Z.nc
20140522_1845Z.nc
20140522_1900Z.nc
20140522_1915Z.nc
20140522_1930Z.nc
20140522_1945Z.nc
20140522_2015Z.nc
20140522_2030Z.nc
20140522_2100Z.nc
20140522_2130Z.nc
20140522_2145Z.nc
20140522_2200Z.nc
20140522_2215Z.nc
20140522_2230Z.nc
20140523_1630Z.nc
20140523_1645Z.nc
20140523_1700Z.nc
20140523_1715Z.nc
20140523_1730Z.nc
20140523_1745Z.nc
20140523_1800Z.nc
20140523_1830Z.nc
20140523_1845Z.nc
20140523_1900Z.nc
20140523_1915Z.nc
20140523_1930Z.nc
20140523_1945Z.nc
20140523_2000Z.nc
20140523_2030Z.nc
20140523_2100Z.nc
20140523_2130Z.nc
20140523_2